In [1]:
from qdrant_client import QdrantClient
from qdrant_client.models import Distance, VectorParams

/home/tomek/simple-rag/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
client = QdrantClient(url="http://localhost:6333")

In [3]:
search_result = client.query_points(
    collection_name="test_collection",
    query=[0.2, 0.1, 0.9, 0.7],
    with_payload=False,
    limit=3
).points

print(search_result)


[ScoredPoint(id=4, version=0, score=1.362, payload=None, vector=None, shard_key=None, order_value=None), ScoredPoint(id=1, version=0, score=1.273, payload=None, vector=None, shard_key=None, order_value=None), ScoredPoint(id=3, version=0, score=1.208, payload=None, vector=None, shard_key=None, order_value=None)]


In [7]:
from qdrant_client.models import Filter, FieldCondition, MatchValue

search_result = client.query_points(
    collection_name="test_collection",
    query=[0.2, 0.1, 0.9, 0.7],
    query_filter=Filter(
        must=[FieldCondition(key="city", match=MatchValue(value="London"))]
    ),
    with_payload=True,
    limit=3,
).points

print(search_result)


[ScoredPoint(id=2, version=0, score=0.871, payload={'city': 'London'}, vector=None, shard_key=None, order_value=None)]


In [6]:
from qdrant_client import models
import ollama

In [8]:
collection_name = "knowledge_base"
EMBEDDING_MODEL="qllama/bge-small-en-v1.5:latest"
LANGUAGE_MODEL = "llama3.2"

In [29]:
client.get_collection(collection_name="knowledge_base").points_count

8

In [20]:
client.get_collections()

CollectionsResponse(collections=[CollectionDescription(name='knowledge_base2'), CollectionDescription(name='buka'), CollectionDescription(name='test_collection'), CollectionDescription(name='knowledge_base')])

In [ ]:
client.create_collection(
    collection_name=collection_name,
    vectors_config=models.VectorParams(size=384, distance=models.Distance.COSINE)
)


True

In [21]:
documents = [
    "Qdrant is a vector database & vector similarity search engine. It deploys as an API service providing search for the nearest high-dimensional vectors. With Qdrant, embeddings or neural network encoders can be turned into full-fledged applications for matching, searching, recommending, and much more!",
    "Docker helps developers build, share, and run applications anywhere — without tedious environment configuration or management.",
    "PyTorch is a machine learning framework based on the Torch library, used for applications such as computer vision and natural language processing.",
    "MySQL is an open-source relational database management system (RDBMS). A relational database organizes data into one or more data tables in which data may be related to each other; these relations help structure the data. SQL is a language that programmers use to create, modify and extract data from the relational database, as well as control user access to the database.",
    "NGINX is a free, open-source, high-performance HTTP server and reverse proxy, as well as an IMAP/POP3 proxy server. NGINX is known for its high performance, stability, rich feature set, simple configuration, and low resource consumption.",
    "FastAPI is a modern, fast (high-performance), web framework for building APIs with Python 3.7+ based on standard Python type hints.",
    "SentenceTransformers is a Python framework for state-of-the-art sentence, text and image embeddings. You can use this framework to compute sentence / text embeddings for more than 100 languages. These embeddings can then be compared e.g. with cosine-similarity to find sentences with a similar meaning. This can be useful for semantic textual similar, semantic search, or paraphrase mining.",
    "The cron command-line utility is a job scheduler on Unix-like operating systems. Users who set up and maintain software environments use cron to schedule jobs (commands or shell scripts), also known as cron jobs, to run periodically at fixed times, dates, or intervals.",
]

In [22]:
document_embeddings = []
for idx, document in enumerate(documents):
    embedding_response = ollama.embeddings(
        model=EMBEDDING_MODEL,  # "qllama/bge-small-en-v1.5:latest"
        prompt=document
    )
    document_embeddings.append({
        'id': idx,
        'vector': embedding_response['embedding'],
        'payload': {'document': document}
    })

In [23]:
client.upsert(
    collection_name=collection_name,
    points=[
        models.PointStruct(
            id=item['id'],
            vector=item['vector'],
            payload=item['payload'],
        )
        for item in document_embeddings
    ],
)

UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

In [9]:
prompt = """
What tools should I need to use to build a web service using vector embeddings for search?
"""

In [10]:
stream = ollama.chat(
    model=LANGUAGE_MODEL,
    messages=[
        {"role": "user", "content": prompt},
    ],
    stream=True,
)
print("Chatbot response:")
for chunk in stream:
    print(chunk["message"]["content"], end="", flush=True)
print("\n")

Chatbot response:
Building a web service using vector embeddings for search requires several tools and technologies. Here's a list of some of the key ones:

**Frontend**

1. **Web Framework**: Choose a suitable web framework such as React, Angular, or Vue.js to build the user interface and handle HTTP requests.
2. **JavaScript Library**: Utilize JavaScript libraries like TensorFlow.js (for vector embedding computation) or D3.js (for visualization).

**Backend**

1. **Programming Language**: Select a programming language such as Python, Java, or C++ to write the backend logic.
2. **Web Framework**: Use a web framework like Flask (Python), Spring Boot (Java), or Express.js (Node.js) to build the RESTful API.

**Vector Embeddings**

1. **Library**: Choose a library that provides vector embedding functionality, such as:
	* TensorFlow.js
	* PyTorch (for Python)
	* Weights & Biases (for Python and other languages)
2. **Model Architecture**: Select a suitable model architecture for your speci

In [16]:
prompt

'\nWhat tools should I need to use to build a web service using vector embeddings for search?\n'

In [12]:
# Generate embedding using ollama
embedding_response = ollama.embeddings(
    model=EMBEDDING_MODEL,
    prompt=prompt
)

In [75]:
# embedding_response['embedding']

In [17]:
models.Document(text=prompt, model=EMBEDDING_MODEL)

Document(text='\nWhat tools should I need to use to build a web service using vector embeddings for search?\n', model='qllama/bge-small-en-v1.5:latest', options=None)

In [26]:
results = client.query_points(
    collection_name=collection_name,
    query=embedding_response['embedding'],
    limit=5
)

In [30]:
results.points

[ScoredPoint(id=7, version=0, score=1.0, payload={'document': 'The cron command-line utility is a job scheduler on Unix-like operating systems. Users who set up and maintain software environments use cron to schedule jobs (commands or shell scripts), also known as cron jobs, to run periodically at fixed times, dates, or intervals.'}, vector=None, shard_key=None, order_value=None),
 ScoredPoint(id=3, version=0, score=0.58223015, payload={'document': 'MySQL is an open-source relational database management system (RDBMS). A relational database organizes data into one or more data tables in which data may be related to each other; these relations help structure the data. SQL is a language that programmers use to create, modify and extract data from the relational database, as well as control user access to the database.'}, vector=None, shard_key=None, order_value=None),
 ScoredPoint(id=1, version=0, score=0.55714494, payload={'document': 'Docker helps developers build, share, and run appli